# Импорт библиотек

In [1]:
import json
import datetime
import csv
import time
import re
try:
    from urllib.request import urlopen, Request
except ImportError:
    from urllib2 import urlopen, Request
import pandas as pd
import numpy as np

# Получение прав

In [2]:
app_id = "380758355733546"
app_secret = "f88ae3a8bd0bd35f401affadaed3527f" 
access_token = app_id + "|" + app_secret

page_id = 'banki.ru'
#page_id = 'hillaryclinton'


# Предотвращение ошибки

In [3]:
def request_until_succeed(url):
    req = Request(url)
    success = False
    while success is False:
        try: 
            response = urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()


# Получение данных-1

In [4]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,created_time,id,likes.limit(1).summary(true),comments.limit(1).summary(true),\
    reactions.limit(0).summary(true), shares&limit=%s&access_token=%s" % (num_statuses, access_token) 
    url = base + node + parameters
    
    data = json.loads(request_until_succeed(url))
    
    return data

# Получение данных-2

In [5]:
def processFacebookPageFeedStatus(status):
        
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode("utf-8")
    
        
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') 
    
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    return (status_id, status_message,
           status_published, num_likes, num_comments, num_shares)

# Запись данных

In [ ]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    
    with open('%s_facebook.csv' % page_id, 'w', newline="", encoding='utf8') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   
        scrape_starttime = datetime.datetime.now()
        
        print("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                num_processed += 1
                if num_processed % 1000 == 0:
                    print ("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))
                    
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
       
        print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))

scrapeFacebookPageFeedStatus(page_id, access_token)


# Создание итоговой таблицы

In [ ]:
Clinton["candidate"] = "Clinton"
Trump["candidate"] = "Trump"
df = pd.concat([Clinton, Trump])

# Анализ настроений

In [ ]:
from textblob import TextBlob
df["sentiment"] = 0

for i in range(0,df.shape[0]):
    if pd.isnull(df.loc[i,"status_message"])==False:
        blob = TextBlob(df.loc[i,"status_message"])
        df.loc[i,"sentiment"] = blob.sentiment.polarity
    else:
        df.loc[i,"sentiment"] = 0

# Создание итогового файла

In [ ]:
writer = pd.ExcelWriter('fb_elections.xlsx')
df.to_excel(writer, index=False)
writer.save()